In [21]:
import getpass
import os

OPEN_AI_KEY = getpass.getpass("OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = OPEN_AI_KEY

OpenAI Key: ··········


In [ ]:
!pip install -U --quiet langgraph "langchain[openai]" langchain-community langchain_text_splitters
!pip install --quiet langchain-openai
!pip install faiss-cpu --quiet
!pip install gradio_client==0.2.10
!pip install gradio==3.38.0

# Step1 : Load Web Page
# Step 2: Chunk the Document ( Chunk size and overlap)

In [28]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/"
]

docs = [WebBaseLoader(url).load() for url in urls]

#print(f"Loaded data from {len(urls)} URLs.")

doc_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

texts = text_splitter.split_documents(doc_list)

In [ ]:
docs[0][0].page_content.strip()[:1000]

In [ ]:
texts[0].page_content.strip()

# Step3 : Create FAISS Vector DB with Embeddings (In memory)

# Step4 : Create Retriever

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.tools.retriever import create_retriever_tool

# create embeddings instance
embeddings = OpenAIEmbeddings()

# build vectorstore from your loaded docs
vectorstore = FAISS.from_documents(texts, embeddings)

retriver = vectorstore.as_retriever()

# Step5:  Create History Aware retriever - Retrieve1
## For creating Contextualize Question from input question using History

### Define Prompt for creating Contextualize question . This should not answer input question.

### Create History Aware Retriever using LLM & retriever , contextualize prompt

In [33]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o")

contextualize_q_system_prompt = (
    "Give the chat history and user latest quesiton which might reference context in the chat history, formulate a standalone question "
    "which can be understood . Without the chat history, DO NOT answer the question, just reformulate it if needed and otherwise return it as is"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

history_awere_retriever = create_history_aware_retriever(
    llm,retriver,contextualize_q_prompt
)

# Step6: Create Final Retriever - Retriever2

In [34]:
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

# Incorporate the retriever into a question-answering chain.

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to only answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

question_answering_chain = create_stuff_documents_chain(llm,qa_prompt)

#History

history_rag_chain = create_retrieval_chain(history_awere_retriever,question_answering_chain)

# contextualize_q_prompt | llm1 | retriever | qa_prompt | llm2

# Step7: Create Gradio based Chatbot with RAG Chain to answer  

In [35]:
from langchain_core.messages import AIMessage , HumanMessage
import gradio as gr

def predict(input, chat_history):
  history_for_llm = []
  for human, ai in chat_history:
    history_for_llm.extend(
        [
            HumanMessage(content=human),
            AIMessage(content=ai)
            ]
        )
  aimsg = history_rag_chain.invoke({"input": input , "chat_history": history_for_llm})
  return aimsg["answer"]

demo = gr.ChatInterface(predict,
                        chatbot=gr.Chatbot(height=200),
                        textbox=gr.Textbox(placeholder="Hi I am your virtual assistent, How I can help you today?", container=False, scale=7),
                        title="blogQABot",
                        theme="soft",
                        examples=["what is Task Decomposition?"],
                        retry_btn=None,
                        undo_btn="Delete Previous")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>